In [ ]:
%pip install langchainhub 
%pip isntall bs4
%pip install openllm
%pip install trulens-eval==0.25.1
%pip install accelerator
%pip install litellm
%pip install "ipython>=8.12.0" "ipywidgets>=8.0.6"
%pip install "ipython"
%pip install unstructured    

In [ ]:
#Do the migration
import os
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.pgvector import PGVector
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from llama_index.core import SimpleDirectoryReader, StorageContext
from sqlalchemy import make_url
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core import Settings

In [ ]:
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_DB = os.getenv('POSTGRES_DB')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')


CONNECTION_STRING = f'postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}'

In [ ]:
##################### EXAMPLE EMBEDDINGS ############################
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
text = "This is a test document."
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text, "This is not a test document."])

In [ ]:
documents = SimpleDirectoryReader("./Acts").load_data()

In [ ]:
print((len(documents)))

In [ ]:
#loader = DirectoryLoader("./Acts")
#documents = loader.load()

In [ ]:
url = make_url(CONNECTION_STRING)
Settings.embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
vector_store = PGVectorStore.from_params(
    database=POSTGRES_DB,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="Acts",
    embed_dim=384,  # openai embedding dimension
)

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
text_parser = SentenceSplitter(
    chunk_size=512,
    # separator=" ",
)

In [ ]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [ ]:
print(len(doc_idxs))

In [ ]:
from llama_index.core.schema import TextNode
nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [ ]:
for node in nodes:
    node_embedding = Settings.embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [ ]:
vector_store.add(nodes)

In [ ]:
query_str = "I've been served with an eviction notice by my landlord. How much notice are they required to give me under the law?"

In [ ]:
query_embedding = Settings.embed_model.get_query_embedding(query_str)

In [ ]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [ ]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

In [ ]:
def retrieval(query_str):
    query_embedding = Settings.embed_model.get_query_embedding(query_str)
    query_mode = "default"
    # query_mode = "sparse"
    # query_mode = "hybrid"
    
    vector_store_query = VectorStoreQuery(
        query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
    )
    query_result = vector_store.query(vector_store_query)
    #print(query_result.nodes[0].get_content())
    return query_result.nodes[0].get_content()


In [ ]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
TRULENS_USER = os.getenv('TRULENS_USER')
TRULENS_PASSWORD = os.getenv('TRULENS_PASSWORD')
TRULENS_DB = os.getenv('TRULENS_DB')
TRULENS_PORT = os.getenv('TRULENS_PORT')
TRULENS_HOST = os.getenv('TRULENS_HOST')

TRULENS_CONNECTION_STRING = f'postgresql+psycopg2://{TRULENS_USER}:{TRULENS_PASSWORD}@{TRULENS_HOST}:{TRULENS_PORT}/{TRULENS_DB}'
tru = Tru(database_url=TRULENS_CONNECTION_STRING)

In [ ]:
tru.reset_database()

In [ ]:
import openllm

llm = openllm.LLM(
    "google/flan-t5-small",
    temperature=0.94,
    repetition_penalty=1.2,

)

In [ ]:
import asyncio

import nest_asyncio



nest_asyncio.apply()


async def main(query):
  previous_texts = ''  
  async for gen in llm.generate_iterator(query, max_new_tokens=128):
      print(gen.outputs[0].text, flush=True, end='')
      previous_texts += gen.outputs[0].text
  return previous_texts


asyncio.run(main("what is tenancy act. Please explain"))

In [ ]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return retrieval(query)

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        messages=f"We have provided context information below. \n" \
            f"---------------------\n" \
            f"{context_str}" \
            f"\n---------------------\n" \
            f"Given this information, please answer the question: {query}"
        print(messages)
        completion = asyncio.run(main(messages))
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

In [ ]:

rag.query("When was the University of Washington founded?")



In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    #feedbacks = [f_groundedness, f_answer_relevance, f_context_relevance]
    )


In [ ]:
with tru_rag as recording:
    rag.query("I've been served with an eviction notice by my landlord. How much notice are they required to give me under the law?")

In [ ]:
from ipywidgets import Button, HBox, VBox
from IPython.display import display

thumbs_up_button = Button(description='👍')
thumbs_down_button = Button(description='👎')

human_feedback = None

def on_thumbs_up_button_clicked(b):
    global human_feedback
    human_feedback = 1

def on_thumbs_down_button_clicked(b):
    global human_feedback
    human_feedback = 0

thumbs_up_button.on_click(on_thumbs_up_button_clicked)
thumbs_down_button.on_click(on_thumbs_down_button_clicked)

HBox([thumbs_up_button, thumbs_down_button])

In [ ]:
record = recording.get()
print(human_feedback)
tru.add_feedback(
    name="Human Feedack",
    record_id=record.record_id,
    app_id=tru_rag.app_id,
    result=human_feedback
)

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.tail(5)

In [ ]:

tru.get_leaderboard(app_ids=["RAG v1"])

In [ ]:
tru.run_dashboard(address='0.0.0.0', force=True)

In [ ]:


tru.get_leaderboard(app_ids=[tru_rag.app_id])
